# Python/R interoperability in Jupyter Notebooks

In this notebook, we demonstrate how to use functionalities in the [`rpy2`](https://rpy2.github.io/) package to use R code within python notebooks and we show examples for conversions between single-cell data object containers across languages.

In [1]:
import pandas as pd
import numpy as np
import scanpy as sc
import anndata 

Let's set up `rpy2` to use R functions and chunks within this python notebook and conversion between `anndata` and `SingleCellExperiment` objects with [`anndata2ri`](https://github.com/theislab/anndata2ri). 

In [2]:
import anndata2ri
anndata2ri.activate() ## activate the conversion before you load the extension
%load_ext rpy2.ipython

In [3]:
import rpy2.rinterface_lib.callbacks
import logging

Let's start by loading one of our anndata objects

In [6]:
data_dir = '/home/jovyan/mount/gdrive/sc-multiomics-course-2021/processed_data/input/gr2_matched_vertical//'

In [9]:
adata = sc.read_h5ad("{d}/Multiome_RNA_clean_SCE.h5ad".format(d=data_dir))

In [11]:
adata

AnnData object with n_obs × n_vars = 8981 × 34104
    obs: 'Sample.ID', 'Sample.Age', 'Sample.Batch', 'Cell.Barcode', 'RNA.Counts', 'RNA.Features', 'Dissociation.ID', 'percentMT', 'percentRibo', 'CR_Estimated.number.of.cells', 'clusterID', 'cluster_name', 'cluster_name_long', 'Assay', 'in_GluN_trajectory'
    var: 'gene_id', 'gene_type', 'gene_name'

### Basic rpy2 extension usage

With the `rpy2` extension for jupyter notebooks, we can use R by starting any chunk with `%%R`

In [13]:
%%R
my.string <- "hello, I'm using R!"
print(my.string)

[1] "hello, I'm using R!"


We can move objects between python and R using the `-i` and `-o` options 

In [15]:
example_df = adata.obs.head()
example_df

,Sample.ID,Sample.Age,Sample.Batch,Cell.Barcode,RNA.Counts,RNA.Features,Dissociation.ID,percentMT,percentRibo,CR_Estimated.number.of.cells,clusterID,cluster_name,cluster_name_long,Assay,in_GluN_trajectory
hft_ctx_w21_dc1r3_r1_AAACAGCCAGCAATAA,hft_ctx_w21_dc1r3_r1,pcw21,b2020_11,AAACAGCCAGCAATAA,2494,1435,A,0.000796,0.017516,3534,c10,GluN5,Glutamatergic neuron 5,Multiome_RNA,True
hft_ctx_w21_dc1r3_r1_AAACAGCCAGCTCATA,hft_ctx_w21_dc1r3_r1,pcw21,b2020_11,AAACAGCCAGCTCATA,2257,1288,A,0.012798,0.033098,3534,c2,IN1,Interneuron 1,Multiome_RNA,False
hft_ctx_w21_dc1r3_r1_AAACATGCACGTTACA,hft_ctx_w21_dc1r3_r1,pcw21,b2020_11,AAACATGCACGTTACA,3485,1632,A,0.031098,0.039372,3534,c0,nIPC/GluN1,Neuronal intermediate progenitor cell/Glutamat...,Multiome_RNA,True
hft_ctx_w21_dc1r3_r1_AAACATGCATAAACCT,hft_ctx_w21_dc1r3_r1,pcw21,b2020_11,AAACATGCATAAACCT,2266,1247,A,0.012319,0.021557,3534,c4,IN2,Interneuron 2,Multiome_RNA,False
hft_ctx_w21_dc1r3_r1_AAACATGCATTTAAGC,hft_ctx_w21_dc1r3_r1,pcw21,b2020_11,AAACATGCATTTAAGC,2741,1495,A,0.012341,0.020690,3534,c4,IN2,Interneuron 2,Multiome_RNA,False


In [19]:
%%R -i example_df
head(example_df) 
## Note that R will try to print full data.frames and it will be hard 
## to stop if printing thousands of lines! 

                                                 Sample.ID Sample.Age
hft_ctx_w21_dc1r3_r1_AAACAGCCAGCAATAA hft_ctx_w21_dc1r3_r1      pcw21
hft_ctx_w21_dc1r3_r1_AAACAGCCAGCTCATA hft_ctx_w21_dc1r3_r1      pcw21
hft_ctx_w21_dc1r3_r1_AAACATGCACGTTACA hft_ctx_w21_dc1r3_r1      pcw21
hft_ctx_w21_dc1r3_r1_AAACATGCATAAACCT hft_ctx_w21_dc1r3_r1      pcw21
hft_ctx_w21_dc1r3_r1_AAACATGCATTTAAGC hft_ctx_w21_dc1r3_r1      pcw21
                                      Sample.Batch     Cell.Barcode RNA.Counts
hft_ctx_w21_dc1r3_r1_AAACAGCCAGCAATAA     b2020_11 AAACAGCCAGCAATAA       2494
hft_ctx_w21_dc1r3_r1_AAACAGCCAGCTCATA     b2020_11 AAACAGCCAGCTCATA       2257
hft_ctx_w21_dc1r3_r1_AAACATGCACGTTACA     b2020_11 AAACATGCACGTTACA       3485
hft_ctx_w21_dc1r3_r1_AAACATGCATAAACCT     b2020_11 AAACATGCATAAACCT       2266
hft_ctx_w21_dc1r3_r1_AAACATGCATTTAAGC     b2020_11 AAACATGCATTTAAGC       2741
                                      RNA.Features Dissociation.ID   percentMT
hft_ctx_w21_dc1r3_r1_AAACAG

In [21]:
%%R -o example_df_2
example_df_2 <- example_df[0:3,]

In [22]:
example_df_2

,Sample.ID,Sample.Age,Sample.Batch,Cell.Barcode,RNA.Counts,RNA.Features,Dissociation.ID,percentMT,percentRibo,CR_Estimated.number.of.cells,clusterID,cluster_name,cluster_name_long,Assay,in_GluN_trajectory
hft_ctx_w21_dc1r3_r1_AAACAGCCAGCAATAA,hft_ctx_w21_dc1r3_r1,pcw21,b2020_11,AAACAGCCAGCAATAA,2494,1435,A,0.000796,0.017516,3534,c10,GluN5,Glutamatergic neuron 5,Multiome_RNA,1
hft_ctx_w21_dc1r3_r1_AAACAGCCAGCTCATA,hft_ctx_w21_dc1r3_r1,pcw21,b2020_11,AAACAGCCAGCTCATA,2257,1288,A,0.012798,0.033098,3534,c2,IN1,Interneuron 1,Multiome_RNA,0
hft_ctx_w21_dc1r3_r1_AAACATGCACGTTACA,hft_ctx_w21_dc1r3_r1,pcw21,b2020_11,AAACATGCACGTTACA,3485,1632,A,0.031098,0.039372,3534,c0,nIPC/GluN1,Neuronal intermediate progenitor cell/Glutamat...,Multiome_RNA,1


### Converting AnnData to SingleCellExperiment

The python package [`anndata2ri`](https://github.com/theislab/anndata2ri) implements a RPy2 converter for single-cell data objects. We can use it to convert an `anndata` object in python to a `SingleCellExperiment` object, usable in R...

In [23]:
%%R -i adata
adata

class: SingleCellExperiment 
dim: 34104 8981 
metadata(0):
assays(1): X
rownames(34104): ENSG00000243485 ENSG00000237613 ... ENSG00000198695
  ENSG00000198727
rowData names(3): gene_id gene_type gene_name
colnames(8981): hft_ctx_w21_dc1r3_r1_AAACAGCCAGCAATAA
  hft_ctx_w21_dc1r3_r1_AAACAGCCAGCTCATA ...
  hft_ctx_w21_dc2r2_r2_TTTGTGAAGACAGGCG
  hft_ctx_w21_dc2r2_r2_TTTGTGTTCGTCCTTA
colData names(15): Sample.ID Sample.Age ... Assay in_GluN_trajectory
reducedDimNames(0):
altExpNames(0):


...and back.

In [25]:
%%R -o adata_2
adata_2 <- adata
colData(adata_2)[['new_coldata']] <- 0

In [27]:
adata_2.obs.head()

,Sample.ID,Sample.Age,Sample.Batch,Cell.Barcode,RNA.Counts,RNA.Features,Dissociation.ID,percentMT,percentRibo,CR_Estimated.number.of.cells,clusterID,cluster_name,cluster_name_long,Assay,in_GluN_trajectory,new_coldata
hft_ctx_w21_dc1r3_r1_AAACAGCCAGCAATAA,hft_ctx_w21_dc1r3_r1,pcw21,b2020_11,AAACAGCCAGCAATAA,2494,1435,A,0.000796,0.017516,3534,c10,GluN5,Glutamatergic neuron 5,Multiome_RNA,1,0.0
hft_ctx_w21_dc1r3_r1_AAACAGCCAGCTCATA,hft_ctx_w21_dc1r3_r1,pcw21,b2020_11,AAACAGCCAGCTCATA,2257,1288,A,0.012798,0.033098,3534,c2,IN1,Interneuron 1,Multiome_RNA,0,0.0
hft_ctx_w21_dc1r3_r1_AAACATGCACGTTACA,hft_ctx_w21_dc1r3_r1,pcw21,b2020_11,AAACATGCACGTTACA,3485,1632,A,0.031098,0.039372,3534,c0,nIPC/GluN1,Neuronal intermediate progenitor cell/Glutamat...,Multiome_RNA,1,0.0
hft_ctx_w21_dc1r3_r1_AAACATGCATAAACCT,hft_ctx_w21_dc1r3_r1,pcw21,b2020_11,AAACATGCATAAACCT,2266,1247,A,0.012319,0.021557,3534,c4,IN2,Interneuron 2,Multiome_RNA,0,0.0
hft_ctx_w21_dc1r3_r1_AAACATGCATTTAAGC,hft_ctx_w21_dc1r3_r1,pcw21,b2020_11,AAACATGCATTTAAGC,2741,1495,A,0.012341,0.020690,3534,c4,IN2,Interneuron 2,Multiome_RNA,0,0.0


**Troubleshooting:** sometimes `anndata2ri` and `rpy2` can throw cryptic errors. Often the issue is that some of your anndata slots contain objects of a type that is not recognized by rpy2 (i.e. something different from `pd.DataFrame` or `np.array` or `scipy.sparse.csr_matrix`). Often the culprits are the `adata.obsp` or `adata.uns` slots. Since these are not converted to SingleCellExperiment, the easiest solution is often to delete them before conversion

### Make Seurat object from anndata

Here we convert first to SingleCellExperiment, then make a Seurat object manually. While technically Seurat implements a conversion function [as.Seurat](https://www.rdocumentation.org/packages/Seurat/versions/3.1.4/topics/as.Seurat) this is often hard to debug.

In [31]:
## Clean anndatas for faster conversion
adata2sce = adata.copy()

In [41]:
sc.pp.normalize_total(adata2sce, target_sum=1e4)
sc.pp.log1p(adata2sce)

sc.pp.highly_variable_genes(adata2sce, min_mean=0.02)
sc.pp.pca(adata2sce)
sc.pp.neighbors(adata2sce, n_neighbors=10, n_pcs=30)
sc.tl.umap(adata2sce)

In [43]:
del adata2sce.obsp["distances"]
del adata2sce.obsp["connectivities"]
del adata2sce.uns

In [44]:
%%R -i adata2sce
adata2sce

class: SingleCellExperiment 
dim: 34104 8981 
metadata(0):
assays(1): X
rownames(34104): ENSG00000243485 ENSG00000237613 ... ENSG00000198695
  ENSG00000198727
rowData names(7): gene_id gene_type ... dispersions dispersions_norm
colnames(8981): hft_ctx_w21_dc1r3_r1_AAACAGCCAGCAATAA
  hft_ctx_w21_dc1r3_r1_AAACAGCCAGCTCATA ...
  hft_ctx_w21_dc2r2_r2_TTTGTGAAGACAGGCG
  hft_ctx_w21_dc2r2_r2_TTTGTGTTCGTCCTTA
colData names(15): Sample.ID Sample.Age ... Assay in_GluN_trajectory
reducedDimNames(2): PCA UMAP
altExpNames(0):


In [45]:
%%R
library(Seurat)
library(Matrix)

## Make Seurat object for RNA 

adata_seu <- CreateSeuratObject(counts=assay(adata2sce, "X"), 
                                      meta.data=data.frame(colData(adata2sce)),
                                      project="brain_chromatin")
# Convert dimensionality reductions
adata_seu[["pca"]] <- CreateDimReducObject(embeddings = reducedDim(adata2sce, "PCA"), key = "PCA_", assay = DefaultAssay(adata_seu))
adata_seu[["umap"]] <- CreateDimReducObject(embeddings = reducedDim(adata2sce, "UMAP"), key = "UMAP_", assay = DefaultAssay(adata_seu))

R[write to console]: Warning:
R[write to console]:  No columnames present in cell embeddings, setting to 'PCA_1:50'

R[write to console]: Warning:
R[write to console]:  No columnames present in cell embeddings, setting to 'UMAP_1:2'



In [46]:
%%R
adata_seu

An object of class Seurat 
34104 features across 8981 samples within 1 assay 
Active assay: RNA (34104 features, 0 variable features)
 2 dimensional reductions calculated: pca, umap


**Troubleshooting:** if you get the following error:
```r
Error in as(object = x, Class = "dgCMatrix") : 
  no method or default for coercing “dgRMatrix” to “dgCMatrix”
```
You might need to convert the sparse matrices in your anndata object to column-oriented sparse matrices before you convert to R

### More resources on interoperability

- Using scanpy in R https://theislab.github.io/scanpy-in-R/#content
- Interoperability with Seurat https://satijalab.org/seurat/articles/conversion_vignette.html